# Profitable App Profiles for the App Store and Google Play Markets

**About:** Working for a company that builds Android and iOS mobile apps. The company only builds apps that are free to download and install - main source of revenue consists of in-app ads. This means that for any given app, the revenue is mostly influenced by the number of users who use the app.

**Goal:** Analyze data to help te developers understand what types of apps are likely to attract more users.

## Opening and Exploring Data

In [1]:
from csv import reader

### The Google Play data set ###
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

### The App Store data set ###
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
print(android_header)
print("\n")
print(ios_header)


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [4]:
explore_data(android,0,4,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10841
Number of columns: 13


In [5]:
explore_data(ios,0,4,True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7197
Number of columns: 16


## Deleting Wrong Data

In [6]:
wrong = android[10472] #discussion has indicated this row has an error
print(wrong) #will see error
print('\n')
print(android_header)  #header column


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


There's not category column entry in the incorrect row, so we'll delete it

In [7]:
del android[10472]

## Removing Duplicate Entires: Part One

In [8]:
#checking for duplicates in Google Play data
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print(f"Number of duplicate apps: {len(duplicate_apps)}")
print("\n")
print(f"Examples of duplicate apps: {duplicate_apps[:14]}")
        

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams']


In [9]:
for app in android:
    name = app[0]
    if name == "Facebook":
        print(app)

['Facebook', 'SOCIAL', '4.1', '78158306', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device']
['Facebook', 'SOCIAL', '4.1', '78128208', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device']


Notice that for the two entries for Facebook, one has more reviews than the other. The one with the most reviews should be the most **recent** entry. So we'll delete the other. This logic will be used to remove duplicates within our data.

## Removing Duplicate Entries: Part Two

In [10]:
reviews_max = {} #place for the highest reviews

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews


In [11]:
#remove duplicate rows now
android_clean = [] #new cleaned data set
already_added = [] #just store app names

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)


To remove the suplicate rows I made sure that the only entries we would keep are the ones with the highes number of reviews. So I looped through the data and found the entries that had the same highest number of reviews  as in my reviews_max dictionary. I then put those rows in the android_clean list, while I put the name of the app in the already_added list (to ensure no duplicates again).

## Removing Non-English Apps: Part One

In [12]:
#checks if name of app is entirely in English
def english(name):
    
    for letter in name:
        number = ord(letter) #checks ordinal of character
        if number > 127:
            return False
    return True
    
english("Instachat 😜")

False

## Removing Non-English Apps: Part Two

We need to refine the previous function so it doesn't unneccesarily delete data that's in English but has some characters outside the ASCII range. We'll have the new function return **False** (non-English) if more than three characters fall outside the ASCII range.

In [13]:
#checks if name of app is entirely in English 
def english_new(name):
    count = 0
    for letter in name:
        number = ord(letter) #checks ordinal of character
        if number > 127:
            count += 1
            if count > 3:
                return False
    return True

english_new('爱奇艺PPS -《欢乐颂2》电视剧热播')


False

In [14]:
android_english = [] #english android apps only
apple_english = [] #english apple apps only

for app in android_clean:
    name = app[0]
    if english_new(name) == True:
        android_english.append(app)

for app in ios:
    name = app[1]
    if english_new(name) == True:
        apple_english.append(app)

print(f"The length of our android English only dataset: {len(android_english)}")
print(f"The length of our apple English only dataset: {len(apple_english)}")


The length of our android English only dataset: 9614
The length of our apple English only dataset: 6183


## Isolating the Free Apps

In this last step of data cleaning, we want only the free apps from each data set.

In [15]:
android_free = []
apple_free = []

for app in android_english:
    pay = app[7] 
    if pay == "0":
        android_free.append(app)

for app in apple_english:
    pay = app[4] 
    if pay == "0.0":
        apple_free.append(app)

print(f"The length of our free android dataset: {len(android_free)}")
print(f"The length of our free apple dataset: {len(apple_free)}")

The length of our free android dataset: 8864
The length of our free apple dataset: 3222


## Most Common Apps by Genre: Part One

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.

2. If the app has a good response from users, we develop it further.

3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. 

In [16]:
android_free[0]

['Photo Editor & Candy Camera & Grid & ScrapBook',
 'ART_AND_DESIGN',
 '4.1',
 '159',
 '19M',
 '10,000+',
 'Free',
 '0',
 'Everyone',
 'Art & Design',
 'January 7, 2018',
 '1.0.0',
 '4.0.3 and up']

The 2nd and 10th column for android apps have genre/category. So we'll look at that fro frequency tables.

In [17]:
apple_free[0]

['284882215',
 'Facebook',
 '389879808',
 'USD',
 '0.0',
 '2974676',
 '212',
 '3.5',
 '3.5',
 '95.0',
 '4+',
 'Social Networking',
 '37',
 '1',
 '29',
 '1']

The ios data has their genre in the 12th column.

## Most Common Apps by Genre: Part Two

We'll build two functions we can use to analyze the frequency tables:

* One function to generate frequency tables that show percentages

* Another function we can use to display the percentages in a descending order

In [18]:
def freq_table(dataset,index):
    
    genre = {}
    count = 0
    
    for app in dataset:
        count += 1
        category = app[index]
        if category in genre:
            genre[category] += 1
        elif category not in genre:
            genre[category] = 1
    
    for category in genre:
        percent = (genre[category] / count) * 100
        genre[category] = percent

    return genre


def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])


## Most Common Apps by Genre: Part Three

1. Analyze the frequency table you generated for the prime_genre column of the App Store data set.

2. Analyze the frequency table you generated for the Category and Genres column of the Google Play data set.

In [19]:
#prime_genre for App Store (apple)
display_table(apple_free, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The most common genre for apple users is "Gaming" **(~58%)**, with the runner-up being "Entertainment" **(~8%)**. The general impression that I get is that most of the free English apps are more for entertainment (games, photo and video, social networking, sports, music).

In [20]:
#category for Android
display_table(android_free, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The most common category for android users is "Family" **(~19%)**, with the runner-up being "Games" **(roughly 10%)**. The general impression that I get is that most of the free English apps on the Google Play Store are more for practical purposes (education, shopping, utilities, productivity, lifestyle) compared to the Apple Store.

In [21]:
#genres for Android
display_table(android_free, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Looking at the genres frequency, it also confirms that the Google Play Store has more apps for practical purposes as compared to Apple.

## Most Popular Apps by Genre on the App Store

The frequency tables we analyzed previously showed us that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and fun apps. Now, we'd like to get an idea about the kind of apps with the most users.

In [22]:
frequency = freq_table(apple_free, 11)

for genre in frequency:
    total = 0 #store the sum of user ratings
    len_genre = 0 #store the number of apps specific to each genre
    for app in apple_free:
        genre_app = app[11]
        if genre_app == genre:
            num_rating = float(app[5])
            total = num_rating + total
            len_genre += 1
    
    avg_rating = total/len_genre
    print(f"Average user rating for {genre}: {avg_rating}")



Average user rating for Social Networking: 71548.34905660378
Average user rating for Photo & Video: 28441.54375
Average user rating for Games: 22788.6696905016
Average user rating for Music: 57326.530303030304
Average user rating for Reference: 74942.11111111111
Average user rating for Health & Fitness: 23298.015384615384
Average user rating for Weather: 52279.892857142855
Average user rating for Utilities: 18684.456790123455
Average user rating for Travel: 28243.8
Average user rating for Shopping: 26919.690476190477
Average user rating for News: 21248.023255813954
Average user rating for Navigation: 86090.33333333333
Average user rating for Lifestyle: 16485.764705882353
Average user rating for Entertainment: 14029.830708661417
Average user rating for Food & Drink: 33333.92307692308
Average user rating for Sports: 23008.898550724636
Average user rating for Book: 39758.5
Average user rating for Finance: 31467.944444444445
Average user rating for Education: 7003.983050847458
Average user

The App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

## Most Popular Apps by Genre on the Google Play

In [39]:
frequency = freq_table(android_free, 1)

for category in frequency:
    total = 0 #store the sum of user ratings
    len_category = 0 #store the number of apps specific to each genre
    for app in android_free:
        category_app = app[1]
        if category_app == category:
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    
    avg_installs = total/len_category
    print(f"Average installs for {category}: {avg_installs}")


Average installs for ART_AND_DESIGN: 1986335.0877192982
Average installs for AUTO_AND_VEHICLES: 647317.8170731707
Average installs for BEAUTY: 513151.88679245283
Average installs for BOOKS_AND_REFERENCE: 8767811.894736841
Average installs for BUSINESS: 1712290.1474201474
Average installs for COMICS: 817657.2727272727
Average installs for COMMUNICATION: 38456119.167247385
Average installs for DATING: 854028.8303030303
Average installs for EDUCATION: 1833495.145631068
Average installs for ENTERTAINMENT: 11640705.88235294
Average installs for EVENTS: 253542.22222222222
Average installs for FINANCE: 1387692.475609756
Average installs for FOOD_AND_DRINK: 1924897.7363636363
Average installs for HEALTH_AND_FITNESS: 4188821.9853479853
Average installs for HOUSE_AND_HOME: 1331540.5616438356
Average installs for LIBRARIES_AND_DEMO: 638503.734939759
Average installs for LIFESTYLE: 1437816.2687861272
Average installs for GAME: 15588015.603248259
Average installs for FAMILY: 3695641.8198090694
Aver

Looking at the data, we can see that although there are a large number of installs for entertainment users prefer more practical apps (previous analysis). And since their market isn't as saturated as the Apple Store when looking at the "Gaming" genre for example, it's safe to say they can continue to focus on practical genres. 

Since we want to make a recommendation for both platforms, I would recommend the focus on practical apps. More specifically maybe the **"Books and Reference"** genre specifically. This genre was chosen because of its practicality and a large number of installs on the Google Play store.